In [2]:
import nltk
import os
import emoji
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktTokenizer
import emoji
import pandas as pd
from keywords import ALL_KEYWORDS

In [4]:
# Ensure NLTK resources are available
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


In [9]:
# Load the dataset
df = pd.read_csv("/notebooks/tweets.csv")

In [ ]:
nltk.data.path.append('Desktop/Thesis/nltk_data')

nltk.data.path.append('Desktop/Thesis/stopwords')

# Set the NLTK data path explicitly
nltk_data_path = "/Users/nltk_data"
os.makedirs(nltk_data_path, exist_ok=True)
nltk.data.path.append(nltk_data_path)

# Drop missing values in the 'text' column
df.dropna(subset=['text'], inplace=True)

# Remove duplicate tweets
df.drop_duplicates(subset=['text'], inplace=True)

# Convert text to lowercase
df['text'] = df['text'].str.lower()

# Remove URLs, mentions (@usernames), hashtags, and special characters (except $ for financial context)
df['text'] = df['text'].replace(r'http\S+|www.\S+', '', regex=True)  # Remove URLs
df['text'] = df['text'].replace(r'@\w+|#\w+', '', regex=True)  # Remove mentions and hashtags
df['text'] = df['text'].replace(r'[^\w\s$]', '', regex=True)  # Keep words and '$' symbols
df['text'] = df['text'].replace(r'\s+', ' ', regex=True).str.strip()  # Remove extra spaces

# Tokenization (Explicit)
df['tokens'] = df['text'].apply(word_tokenize)

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# Convert Emojis to Text
df['tokens'] = df['tokens'].apply(lambda x: [emoji.demojize(word) for word in x])

# Part-of-Speech (POS) Tagging
df['pos_tags'] = df['tokens'].apply(pos_tag)

# Lemmatization with POS Tagging
lemmatizer = WordNetLemmatizer()
def lemmatize_with_pos(tagged_tokens):
    pos_map = {'N': 'n', 'V': 'v', 'J': 'a', 'R': 'r'}  # Mapping POS tags to WordNet format
    return [lemmatizer.lemmatize(word, pos=pos_map.get(tag[0], 'n')) for word, tag in tagged_tokens]

df['lemmatized_tokens'] = df['pos_tags'].apply(lemmatize_with_pos)

# Optional: Stemming (Uncomment if needed)
stemmer = PorterStemmer()
df['stemmed_tokens'] = df['lemmatized_tokens'].apply(lambda x: [stemmer.stem(word) for word in x])

# Convert tokens back to text
df['cleaned_text'] = df['lemmatized_tokens'].apply(lambda x: ' '.join(x))

# Save cleaned data
cleaned_file_path = "/notebooks/clean_7days_tweets.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"Preprocessing complete. Cleaned file saved at: {cleaned_file_path}")

Preprocessing complete. Cleaned file saved at: /notebooks/clean_7days_tweets.csv
